In [ ]:
import torch
from torch_dataset import ZootrDataset 
import os
from pytorch_efficientnet import augmented_pred
from tqdm import tqdm
import gc
from src.preds_functions import choose_preds
from skimage import io
from torchvision import transforms

## Set constants

In [46]:
PATH_TO_FRAMES = r"C:\Users\guilh\Git\ZOOTR\Testes2"
PATH_TO_SAVE = r"C:\Users\guilh\Git\ZOOTR\Items grab preds"

## Load model

In [2]:
mdl = torch.load('model.pth', weights_only=False)

## Get predictions

In [ ]:
test_imgs = [os.path.join(PATH_TO_FRAMES, i) for i in os.listdir(PATH_TO_FRAMES) if i.endswith("png")]

test_set = ZootrDataset(test_imgs, False, augmented_pred)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=20, num_workers=0, drop_last=False, shuffle=False)
#Get test loader for video frames

In [5]:
all_preds = torch.empty(0, 2)
for imgs in tqdm(test_loader):
    with torch.no_grad():
        imgs_pred = mdl(imgs)
        all_preds = torch.cat((all_preds, imgs_pred))
    gc.collect()
#Get predictions dataframe for every saved frame

100%|█████████▉| 5720/5721 [46:45<00:00,  2.04it/s]


OSError: Could not find a backend to open `C:\Users\guilh\Git\ZOOTR\Testes2\New Text Document.txt`` with iomode `r`.

In [13]:
preds = torch.nn.Softmax()(all_preds)
#Preds to probs

c:\Users\guilh\Git\ZOOTR\.venv_zootr\Lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [18]:
items_inds = choose_preds(preds)

In [ ]:
counter = 0
for pred_img in range(len(items_inds)):
    image = io.imread(test_imgs[items_inds[pred_img]])
    #transforms.ToPILImage()(image).show() #check the image
    prob = preds[items_inds][counter,1]
    transforms.ToPILImage()(image).save(os.path.join(PATH_TO_SAVE, "Pred_" + 
                                                     str(counter) + "_" + 
                                                     str(prob.item())  + 
                                                     "_"  + ".png"))
    counter += 1
